# Example of validation form with Jupyter notebook

## Introduction

This document presents the different possibilities offered by Jupyter notebooks and the API of the Python module "trustutils" to create validation forms. 

To work properly, the notebooks require a particular environment set up by TRUST.

## Presentation features


This notebook presents examples of use of the main features available.

Three main modules are presented here: 

* methods to manage validation cases with the [<code>run</code> module](#jupyter.run-module);
* methods to plot graphs, curves and generate tables from output files with the [<code>plot</code> module](#jupyter.plot-module);
* methods to create VisIt renderings with the [<code>visit</code> module](#Visitutils-function)

The complete documentation of the tools is available in the sphinx documentation available with trust -index ?

## Auto-Completion

**Tab** pour la complétion des fonctions des différents modules de trustutils. 
**Shift tab** pour accès à la doctring des modules.

## jupyter.run module

Test cases that need to be executed in the form can be added via the API offered by the <code>run</code> module: 

In [ ]:
from trustutils import run

### General parameters

Some classical introduction blocks can be generated automatically

In [ ]:
# declaration of name and (optionally) date 
run.introduction("TRUST team","21/11/2021")
# In case of a very long description, it is better to write directly in a markdown without using this function
run.description("Some description of the case here")
# declaration of the trust version
run.TRUST_parameters("1.8.0")

### Saving the notebook output / Exporting PDF

By default, the output of the notebooks (text, curves and VisIt images) are not saved to avoid polluting the version control system (git). This can be changed using the instruction below.

This option must be activated if you want to export your notebook in PDF format (File -> Download as ...) with the results of the code cells.

In [ ]:
# This allows you to save the notebook outputs. By default, these are deleted when saving.
#run.saveFormOutput()

### Management of Trust Cases

By default, all the following instructions are executed in the build directory. If you need to access this directory explicitly, you can refer to it with: 

In [ ]:
# run.BUILD_DIRECTORY   ## Contains the absolute path of the build directory.

Then, the <code>addCase</code> method allows to create an object of type <code>TRUSTCase</code> which has methods to copy and/or modify the data set.

By default, on the 1st call, this:
* copies the entire <code>src</code> directory into <code>build</code>
* invokes the <code>prepare</code> script if it exists (on new forms, users are encouraged to perform the maneuvers usually done in the prepare directly in a Jupyter Shell cell)

See the <code>run.TRUSTSuite</code> class to control this more finely.

In [ ]:
# At the beginning, or for fine tuning, it can be useful to reset everything to 0. This deletes the build directory, 
# and empties the list of cases to be executed:
run.reset()

# Here we add the case we want to run, specifying the number of processors we want (default value=1)
run.addCase(".","grad_u_transpose_3d.data",nbProcs=1)

# It possible to use addCaseFromTemplate to generate test case from a template data file 
# using a dictionnary to substitute term starting with $
run.addCaseFromTemplate("diffusion.data",directory="coarse",d={"number": 8})

# Another case, by keeping in a variable the TRUSTCase object to be able to manipulate it:
casT = run.addCaseFromTemplate("diffusion.data","fine",{"number": 32})

# We can also use the options copy to manage the file tree in the build and substitute some text 
casS = casT.copy("NouveauNom2.data", directory="NouveauRep")
casS.substitute("Nombre_de_Noeuds 32 32","Nombre_de_Noeuds 16 16")

# The case can then be added to the list to be executed with addCase
run.addCase(casS)

To access the list of test cases that will be executed: 

In [ ]:
run.printCases()

Then the launching of the cases is done with the method <code>runCases</code>. Pour chaque cas :

* the <code>pre_run</code> script is executed, 
* then the test case 
* and finally the <code>post_run</code> script
 
If a test case crashes, the last 20 lines of the .err file are automatically displayed (you can for example try to modify the previous <code>substitute()</code> to make a case crash).

In [ ]:
# Method to run the test cases
run.runCases()
# The 'verbose' option displays the output of TRUST
# This is usually not wanted because it is already in the terminal
#run.runCases(verbose=True)
# The 'preventConcurrent=True' option ensures that the cases will be executed in the order
# in which they have been registered. Otherwise, if the Sserver is active, it will be automatically used.
#run.runCases(preventConcurrent=True)

The dataset can be displayed with:

In [ ]:
casS.dumpDataset(["dom"])  # highlighting a specific word

The method dumpDataset is specific to the file \*.data, to display a general file, we will use the method:

In [ ]:
#run.dumpData('file_name',list_keywords=[ ])

And finally we can display the performance table for all executed test cases:

In [ ]:
# Methods to display the performance table
run.tablePerf()

### Additional functions

The MEDCoupling environment can be automatically loaded with: 

In [ ]:
run.useMEDCoupling()
# Let's test it!
import medcoupling as mc
print(mc.DataArrayDouble([1,2,3]))

Some additional script or a shell command can be executed from the notebook

In [ ]:
run.executeScript("calcul_nb_mailles")
run.executeCommand("cat nb_mailles")

## jupyter.plot module 

### Graph

This module allows to display curves and other graphs from the TRUST probe and output files. It is based on matplotlib. 

In [ ]:
from trustutils import plot
import numpy as np

# Simple graph from point probe
Graph=plot.Graph("Point Probe",size=[15,8])
Graph.addPoint("fine/diffusion_SONDE.son")

# More complex graph
Graph=plot.Graph("Titre 2",size=[15,8])
Graph.addPoint("coarse/diffusion_SONDE.son",label="Name",color="r",marker="--")
Graph.addPoint("fine/diffusion_SONDE.son",label="Name",color="g",marker="-.")

# To add any curve, use the "add" method. 
x=np.linspace(0,2,100)
y=-100*(x-1)*(x-1)+4
Graph.add(x,y,label="Plot")

# The functions add, addPoint and addSegment take as options 
# the arguments of matplotlib.pyplot.plot
y = np.exp(x)
Graph.add(x,y,label="exp",linewidth=3.,marker='s',mfc='none')

# The legend function takes the arguments of matplotlib.pyplot.legend
Graph.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=2)

# Method to change the axis titles.
Graph.label("x","y")

# Method to frame the plot.
# All parameters are not mandatory:
Graph.visu(xmin=0,xmax=1.5,ymin=0,ymax=3)

In [ ]:
from trustutils import plot
import numpy as np

# To plot the variables of the segment type probes at a given time (by default, at the last time step), 
# the graph displays the closest value in the probe file. 
# The compo keyword corresponds to the index to plot for a field with several dimensions (e.g. speed)
Graph=plot.Graph("Segment probe",size=[15,8])
Graph.addSegment("grad_u_transpose_3d_SONDE_V.son",time=0.,compo=1)

# it is possible to add a function Y=f(X,Y) in addSegment and addPoint (can allow to calculate for example an error)

def substraction(x,y):
    return y - np.sin(np.pi*x)

Graph.addSegment("fine/diffusion_TEMPERATURE_X.son",compo=0,label='segment probe',func=substraction)

# Residuals Plot
Graph=plot.Graph("Residuals",size=[15,8])
Graph.addResidu("fine/diffusion.dt_ev",label="Residuals")

# Method to change the axis scales
Graph.scale(yscale='log')

In [ ]:
from trustutils import plot
import numpy as np 

# We can arrange plots disposition with indice variable
Graph=plot.Graph(title="Multiplot",subtitle="coarse probe",nX=2,nY=2)
Graph.addPoint("coarse/diffusion_SONDE.son")

Graph.addPlot([1,0],"fine probe")
Graph.addPoint("fine/diffusion_SONDE.son") 

Graph.addPlot([0,1])
x=np.linspace(-1,1,100)
y=-x*x+10
Graph.add(x,y,label="Plot",title="Fonction",xIndice=0,yIndice=1)


### Writing in a table

In [ ]:
from trustutils import plot
a = plot.Table(["mesh number", r"time", r"$L_{inf}$(error)"])

# If you want to load a file similar to a text file, 
# you can use the method plot.loadText("direction_of_file.txt")
nb_maille = plot.loadText("nb_mailles", index_column=1, nb_column=1, dtype="str", transpose=True)

data = plot.loadText("coarse/diffusion_NORME_LINF.son", skiprows=0)
a.addLigne([[nb_maille[0][1],data[0][-1],data[1][-1]]],"coarse")

data = plot.loadText("fine/diffusion_NORME_LINF.son")
a.addLigne([[nb_maille[0][2],data[0][-1],data[1][-1]]],"intermediary")

data = plot.loadText("fine/diffusion_NORME_LINF.son")
a.addLigne([[nb_maille[0][3],data[0][-1],data[1][-1]]],"fine")


display(a)

## Visitutils function

Visitutils is a module whose purpose is to help the user to obtain VisIt plots (2D or 3D results).

In [ ]:
from trustutils import visit

# Simple command to visualize a field or mesh
visit.showField("fine/diffusion.lata","Pseudocolor","TEMPERATURE_ELEM_dom",plotmesh=False)
visit.showMesh("fine/diffusion.lata","dom")

In [ ]:
# Use Show class to apply more options

fig=visit.Show("fine/diffusion.lata","Pseudocolor","TEMPERATURE_SOM_dom")
fig.addField("fine/diffusion.lata","Pseudocolor","ERREUR_T_ELEM_dom")
# to visualize probes
fig.checkProbe("fine/diffusion_TEMPERATURE_DIAG.son",sizePoint=10,color="black")

# to add a list of viewing options
fig.visuOptions(['no_databaseinfo','no_legend'])

# To display the visu with the initialization visit.Show()
fig.plot()

In [ ]:
from trustutils import visit

# Square Multiplot
fig=visit.Show("fine/diffusion.lata","Pseudocolor","TEMPERATURE_SOM_dom",plotmesh=False,title="multiplot",nY=2,nX=2)
fig.add("fine/diffusion.lata","Mesh","dom",xIndice=0,yIndice=1)
fig.add("coarse/diffusion.lata","Pseudocolor","TEMPERATURE_SOM_dom",plotmesh=False,xIndice=1,yIndice=0)
fig.add("coarse/diffusion.lata","Mesh","dom",xIndice=1,yIndice=1)
fig.plot()

In [ ]:
from trustutils import plot, visit 

# class export_lata_base to export profile or fields from lata files.
tmp=visit.export_lata_base("fine/diffusion.lata","Pseudocolor","ERREUR_T_ELEM_dom","fine")
tmp.maximun() # register value in Maxfine file
tmp=visit.export_lata_base("coarse/diffusion.lata","Pseudocolor","ERREUR_T_ELEM_dom","coarse")
tmp.maximun()  

MaxFine   =plot.loadText("Maxfine"   )
MaxCoarse =plot.loadText("Maxcoarse")

columns=["Max Error"]
t=plot.Table(columns)      
t.addLigne([[MaxFine]],"fine")    
t.addLigne([[MaxCoarse]],"coarse")    
display(t)

In [ ]:
# 3D Case
from trustutils import visit

# option time to select the frame we want to plot
fig=visit.Show("./grad_u_transpose_3d.lata", "Mesh", "dom", plotmesh=True,nY=2,time=0,title="Mesh")
# Possibility to vary the perspective with rotation3D, normal3D, up3D, zoom3D and zoom2D
fig.rotation3D([45,45,45])
fig.visuOptions(["no_axes","no_bounding_box","no_triad"])

# Zoom on the perpective and min, max value options
fig.add("./grad_u_transpose_3d.lata", "Pseudocolor", "PRESSION_PA_ELEM_dom",plotmesh=True,min=5.,max=6.3,yIndice=1,title="Zoom")
fig.normal3D([0.5,0.5,0.5])
fig.up3D([0,0,1])
fig.zoom3D([0.,0.,1.5])
fig.plot()


In [ ]:
fig=visit.Show("./grad_u_transpose_3d.lata", "Pseudocolor", "PRESSION_PA_ELEM_dom",time=0,plotmesh=True)
fig.addField("./grad_u_transpose_3d.lata", "Vector", "VITESSE_SOM_dom",plotmesh=True)
fig.rotation3D([45,45,45])

# Available Options
fig.blackVector()
fig.meshColor("red")

fig.plot()

fig = visit.Show("./grad_u_transpose_3d.lata", "Contour", "PRESSION_PA_ELEM_dom",title="Isovalues")
fig.rotation3D([45,45,45])
fig.meshTrans()
# Reproduction of visit commands
fig.visitCommand("ContourAtts = ContourAttributes()")
fig.visitCommand("ContourAtts.lineWidth = 2")
fig.visitCommand("ContourAtts.wireframe = 1")
fig.visitCommand("ContourAtts.contourNLevels = 15")
fig.visitCommand("SetPlotOptions(ContourAtts)")

fig.plot()

# It is also possible to use the option empty=True to build on an initially empty base
a=visit.Show(empty=True,title="slice2D")
a.visitCommand("OpenDatabase('grad_u_transpose_3d.lata', 0)")
a.visitCommand("DefineVectorExpression('v_proj_obli45', '{VITESSE_X_SOM_dom/2+VITESSE_Z_SOM_dom/2,VITESSE_Y_SOM_dom,VITESSE_X_SOM_dom/2+VITESSE_Z_SOM_dom/2}')")
a.visitCommand("AddPlot('Vector', 'v_proj_obli45', 1, 0)")

# Formation of a slice2D
a.slice(origin=[0,0,0],normal=[1,1,1.],type_op='slice2d')
a.visitCommand("VectorAtts = VectorAttributes()")
a.visitCommand("VectorAtts.useStride = 1")
a.visitCommand("VectorAtts.stride = 1")
a.visitCommand("VectorAtts.autoScale = 0")
a.visitCommand("VectorAtts.scale = 0.3")
a.visitCommand("SetPlotOptions(VectorAtts)")
a.visitCommand("DrawPlots()")
a.plot()


In case of difficulty, the cli (Command Line Interface) command history of a visu is saved in the file tmp_visit.py and can be replayed with <code>visit -cli -s tmp_visit.py</code>

## Jupyter-Widget Function 

<code>trustutils.widget</code> is a module that allows you to create an interactive widget in the notebook.

This can typically be used to fine-tune a visualization. Once all the parameters are set, we can keep in the final notebook only the corresponding <code>visit.Show</code> command to lighten the form (less heavy than the widget, and more portable).

In [ ]:
# To be used only for the finalization of the form, not for its final version!
from trustutils import widget

# alpha Version
widget.interface("./grad_u_transpose_3d.lata","Pseudocolor","PRESSION_PA_ELEM_dom") 